# Funnel Analysis

## Library Import

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import chart_studio.plotly as py

import plotly.figure_factory as ff
from datetime import timedelta
from plotly import graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

## Data Load

데이터 용량이 커서 메모리 용량의 한계로 에러가 발생하므로 nrows를 통해 3천만개의 데이터만 불러옵니다.

In [2]:
df = pd.read_csv('C:/Users/USER/Desktop/Data/2019-Oct.csv', nrows=30000000)

In [3]:
print(df.shape)
df.head()

(30000000, 9)


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [4]:
df.tail()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
29999995,2019-10-22 13:03:26 UTC,view,1004785,2053013555631882655,electronics.smartphone,huawei,270.52,521138476,a61e7319-33b7-48a3-a68c-b7a65320e9a4
29999996,2019-10-22 13:03:26 UTC,view,18000370,2053013558525952589,NaN,handwers,33.44,546750894,431042f4-3489-40b8-aaba-464f46846151
29999997,2019-10-22 13:03:26 UTC,view,1600591,2053013553056579841,computers.peripherals.printer,epson,282.89,525638463,f8d4956c-37fc-4758-88a6-2d855ef9eb8f
29999998,2019-10-22 13:03:26 UTC,view,4804055,2053013554658804075,electronics.audio.headphone,apple,190.20,556936192,fc00b366-b6b6-42db-a486-19bd4bc696cc
29999999,2019-10-22 13:03:26 UTC,view,12711745,2053013553559896355,NaN,goodyear,52.41,515608846,de7b442b-9b38-4983-93c1-13b50d8a61f4


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000000 entries, 0 to 29999999
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
 8   user_session   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 2.0+ GB


- event type에는 view(상품 보기), purchase(구매), cart(장바구니) 단계가 있다.

In [6]:
df['event_type'].unique()

array(['view', 'purchase', 'cart'], dtype=object)

In [10]:
df['event_type'].value_counts()

view        28800064
cart          666595
purchase      533341
Name: event_type, dtype: int64

category_code, brand, user_session에는 결측치가 존재한다.

In [7]:
df.isnull().sum()

event_time             0
event_type             0
product_id             0
category_id            0
category_code    9621665
brand            4218859
price                  0
user_id                0
user_session           1
dtype: int64

한 명의 유저는 여러 개의 user session을 가질 수 있다.

In [8]:
print(df['user_id'].count())
print(df['user_session'].count())

30000000
29999999


## 데이터 전처리

- event time이 UTC(세계 표준 시간)로 되어있다.
- KST(한국 표준 시간)로 변경해줘야 한다.
- UTC -> KST 방법은 `UTC + 9시간`을 해주면 된다.

In [11]:
df['event_time']

0           2019-10-01 00:00:00 UTC
1           2019-10-01 00:00:00 UTC
2           2019-10-01 00:00:01 UTC
3           2019-10-01 00:00:01 UTC
4           2019-10-01 00:00:04 UTC
                     ...           
29999995    2019-10-22 13:03:26 UTC
29999996    2019-10-22 13:03:26 UTC
29999997    2019-10-22 13:03:26 UTC
29999998    2019-10-22 13:03:26 UTC
29999999    2019-10-22 13:03:26 UTC
Name: event_time, Length: 30000000, dtype: object

In [5]:
df['event_time'] = df['event_time'].str[:19]  # 0 ~ 18번째 글자(UTC 문자열을 제외한 년-월-일 시:분:초)
df['event_time'] = pd.to_datetime(df['event_time'])  # datetime 형식으로 변경
df['event_time'] = df['event_time'] + timedelta(hours=9)  # 9시간 더하기

Funnel 분석에 필요한 컬럼만 남겨둔다.

In [6]:
df = df[['user_id', 'user_session', 'event_type', 'event_time']]
df.head()

,user_id,user_session,event_type,event_time
0,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c,view,2019-10-01 09:00:00
1,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc,view,2019-10-01 09:00:00
2,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8,view,2019-10-01 09:00:01
3,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713,view,2019-10-01 09:00:01
4,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d,view,2019-10-01 09:00:04


user session에 존재하는 1개의 결측치는 제거한다.

In [14]:
df.isnull().sum()

user_id         0
user_session    1
event_type      0
event_time      0
dtype: int64

In [7]:
df = df.dropna()

In [17]:
df.sort_values(by=['user_id', 'event_time'])

,user_id,user_session,event_type,event_time
15808538,64078358,6183edb9-f592-4bb8-9913-67bb4711694d,view,2019-10-13 09:13:46
2420043,183503497,884233e8-8b9f-4970-808b-4e1c81f8a5fc,view,2019-10-03 06:43:00
4799279,184265397,2c5d0468-32e1-42f7-9b4e-b17c1d387247,view,2019-10-05 02:44:37
4800257,184265397,2c5d0468-32e1-42f7-9b4e-b17c1d387247,view,2019-10-05 02:45:18
4808012,184265397,2c5d0468-32e1-42f7-9b4e-b17c1d387247,view,2019-10-05 02:50:50
...,...,...,...,...
29999324,562979932,21424260-667b-4ac2-bced-ed63bcad4274,view,2019-10-22 22:02:55
29999440,562979932,4aae84ad-ac6a-4019-9332-f52683268d88,view,2019-10-22 22:03:00
29999437,562979951,0a17259f-ed61-427b-a97e-e7d67d6705d6,view,2019-10-22 22:03:00
29999497,562979966,84eb4c56-1e2c-4331-8ee0-fd1f374f4765,view,2019-10-22 22:03:03


## Funnel 제작

In [8]:
funnel_dict ={
    'view' : 'Step One',
    'cart' : 'Step Two',
    'purchase' : 'Step Three'
}
# 주석에 작성한 코드와 아래 코드는 같은 결과를 나타낸다. from_dict를 사용하면 orient 옵션을 통해 key 값을 index로 사용할 수 있다.
# pd.DataFrame(funnel_dict.items(), columns=['', 'Step']).set_index('')
steps = pd.DataFrame.from_dict(funnel_dict, orient='index', columns=['Step'])
steps

,Step
view,Step One
cart,Step Two
purchase,Step Three


참고! 
사전의 key를 column으로, value를 row로 설정하여 데이터프레임 만드는 방법

In [37]:
pd.DataFrame([funnel_dict])

,View,Cart,Purchase
0,Step One,Step Two,Step Three


- user session과 event type을 기준으로 그루핑한다.
- user id로 그루핑하지 않는 이유는 한 유저는 여러 user session을 가질 수 있기 때문이다.
    - 예) user id가 1번인 사람이 오늘 쇼핑한 것과 내일 쇼핑한 것은 다르게 구분되어야 한다.

In [9]:
# event time이 빨리 일어난 시간을 계산
group = df.groupby(['user_session', 'event_type']).agg(datetime=('event_time', 'min'))
group

datetime
user_session                         event_type                    
00000042-3e3f-42f9-810d-f3d264139c50 view       2019-10-18 19:54:45
00000083-8816-4d58-a9b8-f52f54186edc cart       2019-10-06 20:33:54
                                     purchase   2019-10-06 20:34:30
                                     view       2019-10-06 20:24:45
000003eb-b63e-45d9-9f26-f229057c654a view       2019-10-03 20:28:52
...                                                             ...
fffff51e-c34a-465a-8369-c845117d6b39 view       2019-10-09 02:10:37
fffff6b0-1ade-413d-b9b4-5fb0dd26aa34 view       2019-10-06 01:40:10
fffff79f-a682-4f63-ad94-8364fe3d648f view       2019-10-18 15:42:56
fffffa1b-a6f0-483a-8f4b-30db242889b6 view       2019-10-08 20:32:24
fffffc65-7ce9-435c-8b72-1d9f7062fe77 view       2019-10-11 15:10:23

[7371181 rows x 1 columns]

steps와 group 데이터를 병합한다.

In [10]:
merged = pd.merge(group, steps, left_on='event_type', right_index=True)
merged

,,datetime,Step
user_session,event_type,,
00000042-3e3f-42f9-810d-f3d264139c50,view,2019-10-18 19:54:45,Step One
00000083-8816-4d58-a9b8-f52f54186edc,view,2019-10-06 20:24:45,Step One
000003eb-b63e-45d9-9f26-f229057c654a,view,2019-10-03 20:28:52,Step One
00000a05-fa4e-4486-b5e8-c8bffa63b422,view,2019-10-07 23:16:54,Step One
00000aaa-d774-49bc-9c31-0c9f6e1c2f0a,view,2019-10-19 21:41:17,Step One
...,...,...,...
ffff0570-ea68-4b43-bd2f-26307ff3612d,purchase,2019-10-22 02:25:51,Step Three
ffff2140-07d9-4f3f-9cdc-9a30ec5aaf33,purchase,2019-10-04 18:01:28,Step Three
ffff2e30-9d65-450b-807e-fcd1c1799da1,purchase,2019-10-14 18:19:57,Step Three


pivot을 이용해 funnel을 제작한다.

In [11]:
funnel = merged.reset_index().pivot(index='user_session', columns='Step', values='datetime')
funnel

Step,Step One,Step Three,Step Two
user_session,,,
00000042-3e3f-42f9-810d-f3d264139c50,2019-10-18 19:54:45,NaT,NaT
00000083-8816-4d58-a9b8-f52f54186edc,2019-10-06 20:24:45,2019-10-06 20:34:30,2019-10-06 20:33:54
000003eb-b63e-45d9-9f26-f229057c654a,2019-10-03 20:28:52,NaT,NaT
00000a05-fa4e-4486-b5e8-c8bffa63b422,2019-10-07 23:16:54,NaT,NaT
00000aaa-d774-49bc-9c31-0c9f6e1c2f0a,2019-10-19 21:41:17,NaT,NaT
...,...,...,...
fffff51e-c34a-465a-8369-c845117d6b39,2019-10-09 02:10:37,NaT,NaT
fffff6b0-1ade-413d-b9b4-5fb0dd26aa34,2019-10-06 01:40:10,NaT,NaT
fffff79f-a682-4f63-ad94-8364fe3d648f,2019-10-18 15:42:56,NaT,NaT


Two와 Three 단계의 컬럼 순서가 바뀌어 있어서 재정렬해준다.

In [12]:
funnel = funnel[['Step One', 'Step Two', 'Step Three']]
funnel.head()

Step,Step One,Step Two,Step Three
user_session,,,
00000042-3e3f-42f9-810d-f3d264139c50,2019-10-18 19:54:45,NaT,NaT
00000083-8816-4d58-a9b8-f52f54186edc,2019-10-06 20:24:45,2019-10-06 20:33:54,2019-10-06 20:34:30
000003eb-b63e-45d9-9f26-f229057c654a,2019-10-03 20:28:52,NaT,NaT
00000a05-fa4e-4486-b5e8-c8bffa63b422,2019-10-07 23:16:54,NaT,NaT
00000aaa-d774-49bc-9c31-0c9f6e1c2f0a,2019-10-19 21:41:17,NaT,NaT


Step 대신 단계별 이름으로 변경해준다.

In [13]:
funnel.columns = steps.index
funnel.head()

,view,cart,purchase
user_session,,,
00000042-3e3f-42f9-810d-f3d264139c50,2019-10-18 19:54:45,NaT,NaT
00000083-8816-4d58-a9b8-f52f54186edc,2019-10-06 20:24:45,2019-10-06 20:33:54,2019-10-06 20:34:30
000003eb-b63e-45d9-9f26-f229057c654a,2019-10-03 20:28:52,NaT,NaT
00000a05-fa4e-4486-b5e8-c8bffa63b422,2019-10-07 23:16:54,NaT,NaT
00000aaa-d774-49bc-9c31-0c9f6e1c2f0a,2019-10-19 21:41:17,NaT,NaT


In [14]:
step_values = [
    ['Steps', 'values'],
    ['View', funnel['view'].notnull().sum()],
    ['Cart', funnel['cart'].notnull().sum()],
    ['Purchase', funnel['purchase'].notnull().sum()]
]

step_values

[['Steps', 'values'],
 ['View', 6509602],
 ['Cart', 410052],
 ['Purchase', 451527]]

In [15]:
table = ff.create_table(step_values)
iplot(table)

In [16]:
data = dict(
    values = [i[1] for i in step_values[1:]],
    steps = ['View', 'Cart', 'Purchase']
)

fig = px.funnel(data, x='values', y='steps')
fig.show()

각 단계로 넘어가는 평균 시간을 계산한다.

In [17]:
funnel.head()

,view,cart,purchase
user_session,,,
00000042-3e3f-42f9-810d-f3d264139c50,2019-10-18 19:54:45,NaT,NaT
00000083-8816-4d58-a9b8-f52f54186edc,2019-10-06 20:24:45,2019-10-06 20:33:54,2019-10-06 20:34:30
000003eb-b63e-45d9-9f26-f229057c654a,2019-10-03 20:28:52,NaT,NaT
00000a05-fa4e-4486-b5e8-c8bffa63b422,2019-10-07 23:16:54,NaT,NaT
00000aaa-d774-49bc-9c31-0c9f6e1c2f0a,2019-10-19 21:41:17,NaT,NaT


view에서 cart로 넘어가는 평균 시간은 5분 15초이다.

In [19]:
# view -> cart
np.mean(funnel['cart'] - funnel['view'])

Timedelta('0 days 00:05:15.930409096')

cart에서 purchase로 넘어가는 평균 시간은 2분 37초이다.

In [20]:
# cart -> purchase
np.mean(funnel['purchase'] - funnel['cart'])

Timedelta('0 days 00:02:37.160869586')